## *Imports*

Install necessary packages for emotional classifer.

In [5]:
%pip install tensorflow_hub

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.5 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.8 MB/s eta 0:00:00
Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.8.0.post1 requires numpy<2.0,>=1.0, but you have numpy 2.0.2 which is incompatible.
hazm 0.10.0 requires numpy==1.24.3, but you have numpy 2.0.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow_hub'

In [3]:
df = pd.read_csv("emotions_dataset.csv")

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['sentence'])

sequences = tokenizer.texts_to_sequences(df['sentence'])
padded_sequences = pad_sequences(sequences, maxlen=50, padding="post")

labels = pd.factorize(df['emotion'])[0]  # Convert labels to numerical values

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:



# Load Persian BERT Model (ParsBERT)
preprocess = hub.load("https://tfhub.dev/alirezamsh/parbert-base/1")
bert_encoder = preprocess

# Build Model
input_text = layers.Input(shape=(), dtype=tf.string, name="text")
bert_layer = bert_encoder(input_text)
dense = layers.Dense(128, activation='relu')(bert_layer)
output = layers.Dense(len(set(labels)), activation='softmax')(dense)

model_bert = Model(inputs=input_text, outputs=output)

# Compile
model_bert.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model_bert.fit(df['sentence'], y_train, epochs=5, batch_size=16, validation_data=(df['sentence'], y_test))
